In [8]:
import pandas as pd
import dask
import glob

In [48]:
DIR="D:\experiments\dataEngineering\input_data\drive\\"
drive_file_list=glob.glob(DIR+"*.parquet")
drive_file_list

['D:\\experiments\\dataEngineering\\input_data\\drive\\drive_1557398801351_trip_1557398302922_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet',
 'D:\\experiments\\dataEngineering\\input_data\\drive\\drive_1557398802487_trip_1557398298604_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet',
 'D:\\experiments\\dataEngineering\\input_data\\drive\\drive_1557398803643_trip_1557398315398_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet',
 'D:\\experiments\\dataEngineering\\input_data\\drive\\drive_1557398804837_trip_1557398318847_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet',
 'D:\\experiments\\dataEngineering\\input_data\\drive\\drive_1557398805955_trip_1557398271712_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet',
 'D:\\experiments\\dataEngineering\

In [5]:
df=pd.read_parquet(drive_file_list[0])

In [6]:
df.head()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:00,0.00,50.0,65.13,87.92,150.70,192.88,118.66,95.0,2107.44
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:01,47.49,51.0,72.34,83.78,154.76,196.04,116.69,100.0,2119.17
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:02,57.94,50.0,69.20,87.73,165.50,196.01,105.46,90.0,2112.32
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:03,57.50,48.0,70.83,84.95,157.32,193.31,110.35,90.0,2092.94
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:04,52.29,47.0,70.22,92.90,151.00,189.49,104.25,97.0,2117.87


# 1. Engine Features (file name – engine_features.csv)-
- Grain - every vehicle aggregated at week start date(Monday) for the complete week in YYYY-MM-DD format.
- Sorted - by Vehicle ID and week_start_Date in ascending manner

 

### Hints:

- Convert timezone to PST before any calculations
- All vehicles from drive data should be in the final output even if you do not have specifications (Fill with 0 if specs are not given)
- Active horsepower - Engine load / 255 * Max Torque * RPM / 5252
- Horsepower utilization – Active horsepower / Max Horsepower
- Torque Utilization - calculated as Engine load/ 255
- RPM Utilization – RPM / Maximum horsepower rpm


In [12]:
from pytz import all_timezones
all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [22]:
from datetime import datetime
from pytz import timezone
import pytz

date_format='%m/%d/%Y %H:%M:%S %Z'
date = datetime.now(tz=pytz.utc)
print('Current date & time is:', date.strftime(date_format))

date = date.astimezone(timezone('US/Pacific'))

print('Local date & time is  :', date.strftime(date_format))


Current date & time is: 07/13/2019 11:40:13 UTC
Local date & time is  : 07/13/2019 21:40:13 +10


In [23]:
df.dtypes

vehicle_id                      int64
trip_id                        object
datetime               datetime64[ns]
velocity                      float64
accel_x                       float64
accel_y                       float64
accel_z                       float64
engine_coolant_temp           float64
eng_load                      float64
fuel_level                    float64
iat                           float64
rpm                           float64
dtype: object

In [126]:
df['datetime_pst']=df['datetime'].dt.tz_localize(timezone('US/Pacific'))

In [37]:
df1=df.set_index(pd.DatetimeIndex(df['datetime_pst']))

In [42]:
df1.resample("1W").sum()

,vehicle_id,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
datetime_pst,,,,,,,,,,
2017-01-08 00:00:00-08:00,118875723723,7475446.08,8253072.27,8096575.65,7075760.74,14536345.49,24024498.31,13960620.27,10882016.20,2.465777e+08
2017-01-15 00:00:00-08:00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
2017-01-22 00:00:00-08:00,43107033332,2662936.64,2702067.12,2677161.32,3164741.44,5160321.86,8925576.31,5450412.67,3452172.21,8.934149e+07
2017-01-29 00:00:00-08:00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
2017-02-05 00:00:00-08:00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
2017-02-12 00:00:00-08:00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
2017-02-19 00:00:00-08:00,20949595519,1326783.56,1375525.86,1310233.08,1397330.11,1831740.00,4229009.57,2170770.07,2261085.98,4.369037e+07


In [43]:
df['week_start_date']=df['datetime_pst'].dt.date

In [44]:
df.head()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm,datetime_pst,week_start_date
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:00,0.00,50.0,65.13,87.92,150.70,192.88,118.66,95.0,2107.44,2017-01-06 23:00:00-08:00,2017-01-06
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:01,47.49,51.0,72.34,83.78,154.76,196.04,116.69,100.0,2119.17,2017-01-06 23:00:01-08:00,2017-01-06
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:02,57.94,50.0,69.20,87.73,165.50,196.01,105.46,90.0,2112.32,2017-01-06 23:00:02-08:00,2017-01-06
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:03,57.50,48.0,70.83,84.95,157.32,193.31,110.35,90.0,2092.94,2017-01-06 23:00:03-08:00,2017-01-06
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:04,52.29,47.0,70.22,92.90,151.00,189.49,104.25,97.0,2117.87,2017-01-06 23:00:04-08:00,2017-01-06


In [51]:
vehicle=pd.read_csv(r"D:\experiments\dataEngineering\input_data\vehicle.csv")
vehicle.head()

,vehicle_id,year,make,Model,drivetrain,max_torque,max_horsepower,max_horsepower_rpm,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation
0,1000500,2016,Honda,Civic,2,174,140,6500,1500,1.799,1059,47,16.50,20.00,4,1054.0,5
1,1000501,2016,Jeep,Compass,4,350,171,3750,1250,1.956,1059,60,17.10,21.20,3,NaN,4
2,1000502,2016,Hyundai,Creta,2,260,126,4000,1500,1.582,1059,55,19.67,24.10,4,1054.0,3
3,1000503,2016,Skoda,Superb,2,250,177,5100,1750,1.798,1059,66,14.67,23.30,3,NaN,2
4,1000504,2017,Volkswagen,Passat,4,350,174,3600,1500,1.968,1059,66,17.42,20.43,4,NaN,4


In [61]:
df_v=df.merge(vehicle,how='left',on='vehicle_id')

In [58]:
df['vehicle_id'].value_counts()

1000518    20081
1000502    18836
1000517    14922
1000503    14243
1000511    13998
1000515    13225
1000513    12493
1000505    12268
1000519    11485
1000510    10516
1000506    10052
1000504     9964
1000509     8072
1000516     6180
1000512     4168
1000507     2336
Name: vehicle_id, dtype: int64

In [62]:
df_v['vehicle_id'].value_counts()

1000518    20081
1000502    18836
1000517    14922
1000503    14243
1000511    13998
1000515    13225
1000513    12493
1000505    12268
1000519    11485
1000510    10516
1000506    10052
1000504     9964
1000509     8072
1000516     6180
1000512     4168
1000507     2336
Name: vehicle_id, dtype: int64

### Active horsepower - Engine load / 255 * Max Torque * RPM / 5252

In [63]:
df_v['active_horse_pow']=(df_v['eng_load']/255)*df_v['max_torque']*df_v['rpm']/5252

In [68]:
df_v[~df_v['active_horse_pow'].isnull()]#.sum()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation,active_horse_pow
4250,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:00,0.00,90.76,65.53,46.0,90.00,206.62,164.48,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,127.357503
4251,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:01,73.61,92.94,73.38,48.0,91.00,203.66,159.68,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,125.352345
4252,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:02,72.70,88.72,75.97,42.0,98.00,206.60,170.15,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,127.368007
4253,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:03,87.50,85.50,67.95,36.0,100.00,198.50,158.91,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,122.133689
4254,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:04,34.05,91.79,69.48,43.0,106.00,200.86,165.75,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,124.390841
4255,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:05,66.26,85.61,75.47,41.0,94.00,202.04,168.42,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,124.544718
4256,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:06,64.76,79.72,72.95,50.0,99.00,202.93,165.25,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,124.690292
4257,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:07,80.65,80.45,75.98,50.0,98.00,203.02,155.38,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,125.160952
4258,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:08,59.89,93.80,79.89,48.0,108.00,207.69,162.95,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,127.698195
4259,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:09,90.67,76.88,60.13,55.0,94.00,198.12,173.81,...,1800.0,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,122.346045


### Horsepower utilization – Active horsepower / Max Horsepower

In [71]:
df_v['horse_pow']=df_v['active_horse_pow']/df_v['active_horse_pow'].max()
df_v[~df_v['horse_pow'].isnull()]

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation,active_horse_pow,horse_pow
4250,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:00,0.00,90.76,65.53,46.0,90.00,206.62,164.48,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,127.357503,0.846176
4251,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:01,73.61,92.94,73.38,48.0,91.00,203.66,159.68,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,125.352345,0.832854
4252,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:02,72.70,88.72,75.97,42.0,98.00,206.60,170.15,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,127.368007,0.846246
4253,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:03,87.50,85.50,67.95,36.0,100.00,198.50,158.91,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,122.133689,0.811469
4254,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:04,34.05,91.79,69.48,43.0,106.00,200.86,165.75,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,124.390841,0.826465
4255,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:05,66.26,85.61,75.47,41.0,94.00,202.04,168.42,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,124.544718,0.827488
4256,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:06,64.76,79.72,72.95,50.0,99.00,202.93,165.25,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,124.690292,0.828455
4257,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:07,80.65,80.45,75.98,50.0,98.00,203.02,155.38,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,125.160952,0.831582
4258,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:08,59.89,93.80,79.89,48.0,108.00,207.69,162.95,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,127.698195,0.848440
4259,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:09,90.67,76.88,60.13,55.0,94.00,198.12,173.81,...,1.995,1059.0,57.0,21.76,26.32,3.0,NaN,3.0,122.346045,0.812879


### - Torque Utilization - calculated as Engine load/ 255


In [73]:
df_v['torque_util']=df_v['eng_load']/255
df_v[~df_v['torque_util'].isnull()]

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation,active_horse_pow,horse_pow,torque_util
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:00,0.00,50.0,65.13,87.92,150.70,192.88,118.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.756392
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:01,47.49,51.0,72.34,83.78,154.76,196.04,116.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.768784
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:02,57.94,50.0,69.20,87.73,165.50,196.01,105.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.768667
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:03,57.50,48.0,70.83,84.95,157.32,193.31,110.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.758078
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:04,52.29,47.0,70.22,92.90,151.00,189.49,104.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.743098
5,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:05,47.75,53.0,69.60,93.51,158.87,189.37,111.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.742627
6,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:06,57.21,52.0,69.04,87.93,152.10,188.49,107.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.739176
7,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:07,61.39,54.0,66.66,88.50,152.83,189.18,110.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.741882
8,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:08,47.90,50.0,66.17,90.73,165.73,196.08,113.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.768941
9,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:09,54.93,46.0,71.54,86.34,150.75,184.89,111.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.725059


### - RPM Utilization – RPM / Maximum horsepower rpm


In [74]:
df_v['rpm_util']=df_v['rpm']/df_v['max_horsepower_rpm']
df_v[~df_v['rpm_util'].isnull()]

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation,active_horse_pow,horse_pow,torque_util,rpm_util
4250,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:00,0.00,90.76,65.53,46.0,90.00,206.62,164.48,...,57.0,21.76,26.32,3.0,NaN,3.0,127.357503,0.846176,0.810275,0.515938
4251,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:01,73.61,92.94,73.38,48.0,91.00,203.66,159.68,...,57.0,21.76,26.32,3.0,NaN,3.0,125.352345,0.832854,0.798667,0.515195
4252,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:02,72.70,88.72,75.97,42.0,98.00,206.60,170.15,...,57.0,21.76,26.32,3.0,NaN,3.0,127.368007,0.846246,0.810196,0.516030
4253,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:03,87.50,85.50,67.95,36.0,100.00,198.50,158.91,...,57.0,21.76,26.32,3.0,NaN,3.0,122.133689,0.811469,0.778431,0.515015
4254,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:04,34.05,91.79,69.48,43.0,106.00,200.86,165.75,...,57.0,21.76,26.32,3.0,NaN,3.0,124.390841,0.826465,0.787686,0.518370
4255,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:05,66.26,85.61,75.47,41.0,94.00,202.04,168.42,...,57.0,21.76,26.32,3.0,NaN,3.0,124.544718,0.827488,0.792314,0.515980
4256,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:06,64.76,79.72,72.95,50.0,99.00,202.93,165.25,...,57.0,21.76,26.32,3.0,NaN,3.0,124.690292,0.828455,0.795804,0.514317
4257,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:07,80.65,80.45,75.98,50.0,98.00,203.02,155.38,...,57.0,21.76,26.32,3.0,NaN,3.0,125.160952,0.831582,0.796157,0.516030
4258,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:08,59.89,93.80,79.89,48.0,108.00,207.69,162.95,...,57.0,21.76,26.32,3.0,NaN,3.0,127.698195,0.848440,0.814471,0.514653
4259,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:09,90.67,76.88,60.13,55.0,94.00,198.12,173.81,...,57.0,21.76,26.32,3.0,NaN,3.0,122.346045,0.812879,0.776941,0.516900


In [80]:
df_v['torque_util_per']=(df_v['torque_util']/df_v['torque_util'].mean())*100

In [81]:
df_v[~df_v['torque_util_per'].isnull()]

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation,active_horse_pow,horse_pow,torque_util,rpm_util,torque_util_per
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:00,0.00,50.0,65.13,87.92,150.70,192.88,118.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.756392,NaN,94.854371
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:01,47.49,51.0,72.34,83.78,154.76,196.04,116.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.768784,NaN,96.408393
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:02,57.94,50.0,69.20,87.73,165.50,196.01,105.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.768667,NaN,96.393640
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:03,57.50,48.0,70.83,84.95,157.32,193.31,110.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.758078,NaN,95.065836
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:04,52.29,47.0,70.22,92.90,151.00,189.49,104.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.743098,NaN,93.187239
5,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:05,47.75,53.0,69.60,93.51,158.87,189.37,111.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.742627,NaN,93.128226
6,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:06,57.21,52.0,69.04,87.93,152.10,188.49,107.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.739176,NaN,92.695460
7,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:07,61.39,54.0,66.66,88.50,152.83,189.18,110.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.741882,NaN,93.034788
8,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:08,47.90,50.0,66.17,90.73,165.73,196.08,113.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.768941,NaN,96.428064
9,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:09,54.93,46.0,71.54,86.34,150.75,184.89,111.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.725059,NaN,90.925055


![](IMG/engine_features.png)

In [117]:
df_v['ft_torque_util_60pct_s']=df_v[(0.60<=df_v['torque_util'])&(df_v['torque_util']<0.70)]['seconds']
df_v['ft_torque_util_70pct_s']=df_v[(0.70<=df_v['torque_util'])&(df_v['torque_util']<0.80)]['seconds']
df_v['ft_torque_util_80pct_s']=df_v[(0.80<=df_v['torque_util'])&(df_v['torque_util']<0.90)]['seconds']
df_v['ft_torque_util_90pct_s']=df_v[(0.90<=df_v['torque_util'])&(df_v['torque_util']<1)]['seconds']


In [124]:
df_v['ft_rpm_util_50pct_s']=df_v[(0.50<=df_v['rpm_util'])&(df_v['rpm_util']<0.6)]['seconds']
df_v['ft_rpm_util_60pct_s']=df_v[(0.60<=df_v['rpm_util'])&(df_v['rpm_util']<0.7)]['seconds']

In [123]:
df_v['ft_horsepower_util_50pct_s']=df_v[(0.50<=df_v['horse_pow'])&(df_v['horse_pow']<0.60)]['seconds']
df_v['ft_horsepower_util_60pct_s']=df_v[(0.60<=df_v['horse_pow'])&(df_v['horse_pow']<0.70)]['seconds']
df_v['ft_horsepower_util_70pct_s']=df_v[(0.70<=df_v['horse_pow'])&(df_v['horse_pow']<0.80)]['seconds']
df_v['ft_horsepower_util_80pct_s']=df_v[(0.80<=df_v['horse_pow'])&(df_v['horse_pow']<0.90)]['seconds']

df_v[~df_v['ft_horsepower_util_80pct_s'].isnull()]

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,seconds,ft_torque_util_60_pts,ft_torque_util_60pct_s,ft_torque_util_70pct_s,ft_torque_util_80pct_s,ft_torque_util_90pct_s,ft_horsepower_util_50pct_s,ft_horsepower_util_60pct_s,ft_horsepower_util_70pct_s,ft_horsepower_util_80pct_s
4250,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:00,0.00,90.76,65.53,46.00,90.0,206.62,164.48,...,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000
4251,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:01,73.61,92.94,73.38,48.00,91.0,203.66,159.68,...,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
4252,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:02,72.70,88.72,75.97,42.00,98.0,206.60,170.15,...,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000
4253,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:03,87.50,85.50,67.95,36.00,100.0,198.50,158.91,...,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
4254,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:04,34.05,91.79,69.48,43.00,106.0,200.86,165.75,...,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
4255,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:05,66.26,85.61,75.47,41.00,94.0,202.04,168.42,...,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
4256,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:06,64.76,79.72,72.95,50.00,99.0,202.93,165.25,...,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
4257,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:07,80.65,80.45,75.98,50.00,98.0,203.02,155.38,...,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
4258,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:08,59.89,93.80,79.89,48.00,108.0,207.69,162.95,...,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000
4259,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:09,90.67,76.88,60.13,55.00,94.0,198.12,173.81,...,0.000000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000


In [85]:
df_v['seconds']=(df['datetime_pst'].dt.minute/60)

In [93]:
(df_v['torque_util']>0.70)&(0.60<=df_v['torque_util'])

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
          ... 
182809    True
182810    True
182811    True
182812    True
182813    True
182814    True
182815    True
182816    True
182817    True
182818    True
182819    True
182820    True
182821    True
182822    True
182823    True
182824    True
182825    True
182826    True
182827    True
182828    True
182829    True
182830    True
182831    True
182832    True
182833    True
182834    True
182835    True
182836    True
182837    True
182838    True
Name: torque_util, Length: 182839, dtype: bool

In [115]:
df_v['ft_torque_util_60_pts']=df_v[(0.60<=df_v['torque_util'])&(df_v['torque_util']<0.70)]['seconds']

In [98]:
df_v[~df_v['rpm_util'].isnull()]

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,fuel_economy_highway,cylinders,forced_induction,device_generation,active_horse_pow,horse_pow,torque_util,rpm_util,torque_util_per,seconds
4250,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:00,0.00,90.76,65.53,46.0,90.00,206.62,164.48,...,26.32,3.0,NaN,3.0,127.357503,0.846176,0.810275,0.515938,101.611417,0.000000
4251,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:01,73.61,92.94,73.38,48.0,91.00,203.66,159.68,...,26.32,3.0,NaN,3.0,125.352345,0.832854,0.798667,0.515195,100.155750,0.000000
4252,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:02,72.70,88.72,75.97,42.0,98.00,206.60,170.15,...,26.32,3.0,NaN,3.0,127.368007,0.846246,0.810196,0.516030,101.601581,0.000000
4253,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:03,87.50,85.50,67.95,36.0,100.00,198.50,158.91,...,26.32,3.0,NaN,3.0,122.133689,0.811469,0.778431,0.515015,97.618170,0.000000
4254,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:04,34.05,91.79,69.48,43.0,106.00,200.86,165.75,...,26.32,3.0,NaN,3.0,124.390841,0.826465,0.787686,0.518370,98.778769,0.000000
4255,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:05,66.26,85.61,75.47,41.0,94.00,202.04,168.42,...,26.32,3.0,NaN,3.0,124.544718,0.827488,0.792314,0.515980,99.359068,0.000000
4256,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:06,64.76,79.72,72.95,50.0,99.00,202.93,165.25,...,26.32,3.0,NaN,3.0,124.690292,0.828455,0.795804,0.514317,99.796752,0.000000
4257,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:07,80.65,80.45,75.98,50.0,98.00,203.02,155.38,...,26.32,3.0,NaN,3.0,125.160952,0.831582,0.796157,0.516030,99.841012,0.000000
4258,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:08,59.89,93.80,79.89,48.0,108.00,207.69,162.95,...,26.32,3.0,NaN,3.0,127.698195,0.848440,0.814471,0.514653,102.137620,0.000000
4259,1000506,77b52511437f4b78871eb1ef6080daae,2017-01-06 23:00:09,90.67,76.88,60.13,55.0,94.00,198.12,173.81,...,26.32,3.0,NaN,3.0,122.346045,0.812879,0.776941,0.516900,97.431294,0.000000


In [116]:
df_v.groupby("vehicle_id")['ft_torque_util_60_pts'].sum()

vehicle_id
1000502     1.483333
1000503     0.000000
1000504     0.000000
1000505     0.000000
1000506     0.000000
1000507     0.000000
1000509     0.000000
1000510     0.000000
1000511    36.583333
1000512     0.000000
1000513     4.183333
1000515     2.216667
1000516     1.050000
1000517     0.000000
1000518     8.933333
1000519     0.900000
Name: ft_torque_util_60_pts, dtype: float64

In [128]:
grouped=df_v.groupby("vehicle_id")

In [132]:
for i,j in grouped:
    print(j)

        vehicle_id                           trip_id            datetime  \
49796      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:00   
49797      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:01   
49798      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:02   
49799      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:03   
49800      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:04   
49801      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:05   
49802      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:06   
49803      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:07   
49804      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:08   
49805      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:09   
49806      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:10   
49807      1000502  f3a5b3f11af34cf6b0231467bcd5ca44 2017-02-14 05:00:11   
49808      1

        vehicle_id                           trip_id            datetime  \
22428      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:00   
22429      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:01   
22430      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:02   
22431      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:03   
22432      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:04   
22433      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:05   
22434      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:06   
22435      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:07   
22436      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:08   
22437      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:09   
22438      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:10   
22439      1000507  c398182d90c34132aad863caea0ce1bc 2017-01-06 17:00:11   
22440      1

In [137]:
df_v

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            182829, 182830, 182831, 182832, 182833, 182834, 182835, 182836,
            182837, 182838],
           dtype='int64', length=182839)

In [142]:
df_v['datetime_pst']=pd.DatetimeIndex(df_v['datetime_pst'])
df_v.resample("1w",on='datetime_pst').sum()

,vehicle_id,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm,...,ft_torque_util_60pct_s,ft_torque_util_70pct_s,ft_torque_util_80pct_s,ft_torque_util_90pct_s,ft_horsepower_util_50pct_s,ft_horsepower_util_60pct_s,ft_horsepower_util_70pct_s,ft_horsepower_util_80pct_s,ft_rpm_util_50pct_s,ft_rpm_util_60pct_s
datetime_pst,,,,,,,,,,,,,,,,,,,,,
2017-01-08 00:00:00-08:00,118875723723,7475446.08,8253072.27,8096575.65,7075760.74,14536345.49,24024498.31,13960620.27,10882016.20,2.465777e+08,...,13.650000,31150.300000,18949.616667,6.966667,5589.783333,202.333333,698.316667,3266.133333,7325.266667,0.000000
2017-01-15 00:00:00-08:00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-22 00:00:00-08:00,43107033332,2662936.64,2702067.12,2677161.32,3164741.44,5160321.86,8925576.31,5450412.67,3452172.21,8.934149e+07,...,5.083333,6709.783333,11699.566667,43.350000,2433.283333,0.000000,88.800000,986.466667,178.350000,0.000000
2017-01-29 00:00:00-08:00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-02-05 00:00:00-08:00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-02-12 00:00:00-08:00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-02-19 00:00:00-08:00,20949595519,1326783.56,1375525.86,1310233.08,1397330.11,1831740.00,4229009.57,2170770.07,2261085.98,4.369037e+07,...,36.616667,5759.433333,3634.350000,29.916667,1312.916667,0.000000,377.133333,3172.950000,32.200000,3549.983333


In [156]:
dates = pd.date_range("2019-07-01", "2019-07-31")

# create a data with only 1,2,3 values
data = [3] * int(len(dates))

dates_df = pd.DataFrame(data=data, index=dates)
dates_df.head()


dates_df.loc[[
    pd.Timestamp('2019-07-18'),
    pd.Timestamp('2019-07-15'),
    pd.Timestamp('2019-07-20'),
    pd.Timestamp('2019-07-17'),
]]


,0
2019-07-18,3
2019-07-15,3
2019-07-20,3
2019-07-17,3


In [161]:
from pandas.tseries.offsets import WeekOfMonth,Day

dates_df.resample("W-Mon").sum().dropna()


,0
2019-07-01,3
2019-07-08,21
2019-07-15,21
2019-07-22,21
2019-07-29,21
2019-08-05,6


In [173]:
def vehicle_wise_weekly_agg(engine_feat_agg,agg_cols):
    grouped=engine_feat_agg.groupby('vehicle_id')
    res_df=[]
    for vehicle_id,df in grouped:
        print(df.columns.tolist())
        temp=df.set_index(pd.DatetimeIndex(df['datetime_pst']))
        temp=temp.resample("W-Mon")[agg_cols].sum()
        temp['vehicle_id']=vehicle_id
        temp['week_start_Date']=temp.index.date
        res_df.append(temp)

    return pd.concat(res_df).sort_values(by=['vehicle_id',"week_start_Date"])


In [177]:
df_v['vehicle_id'].value_counts().count()

16

In [163]:
agg_cols=['ft_torque_util_60pct_s','ft_torque_util_70pct_s','ft_torque_util_80pct_s',
    'ft_torque_util_90pct_s','ft_horsepower_util_50pct_s','ft_horsepower_util_60pct_s',
    'ft_horsepower_util_70pct_s','ft_horsepower_util_80pct_s','ft_rpm_util_50pct_s',
    'ft_rpm_util_60pct_s']

In [174]:
df_r=vehicle_wise_weekly_agg(df_v,agg_cols)

['vehicle_id', 'trip_id', 'datetime', 'velocity', 'accel_x', 'accel_y', 'accel_z', 'engine_coolant_temp', 'eng_load', 'fuel_level', 'iat', 'rpm', 'datetime_pst', 'week_start_date', 'year', 'make', 'Model', 'drivetrain', 'max_torque', 'max_horsepower', 'max_horsepower_rpm', 'max_torque_rpm', 'engine_displacement', 'fuel_type', 'fuel_tank_capacity', 'fuel_economy_city', 'fuel_economy_highway', 'cylinders', 'forced_induction', 'device_generation', 'active_horse_pow', 'horse_pow', 'torque_util', 'rpm_util', 'torque_util_per', 'seconds', 'ft_torque_util_60_pts', 'ft_torque_util_60pct_s', 'ft_torque_util_70pct_s', 'ft_torque_util_80pct_s', 'ft_torque_util_90pct_s', 'ft_horsepower_util_50pct_s', 'ft_horsepower_util_60pct_s', 'ft_horsepower_util_70pct_s', 'ft_horsepower_util_80pct_s', 'ft_rpm_util_50pct_s', 'ft_rpm_util_60pct_s']
['vehicle_id', 'trip_id', 'datetime', 'velocity', 'accel_x', 'accel_y', 'accel_z', 'engine_coolant_temp', 'eng_load', 'fuel_level', 'iat', 'rpm', 'datetime_pst', 'wee

['vehicle_id', 'trip_id', 'datetime', 'velocity', 'accel_x', 'accel_y', 'accel_z', 'engine_coolant_temp', 'eng_load', 'fuel_level', 'iat', 'rpm', 'datetime_pst', 'week_start_date', 'year', 'make', 'Model', 'drivetrain', 'max_torque', 'max_horsepower', 'max_horsepower_rpm', 'max_torque_rpm', 'engine_displacement', 'fuel_type', 'fuel_tank_capacity', 'fuel_economy_city', 'fuel_economy_highway', 'cylinders', 'forced_induction', 'device_generation', 'active_horse_pow', 'horse_pow', 'torque_util', 'rpm_util', 'torque_util_per', 'seconds', 'ft_torque_util_60_pts', 'ft_torque_util_60pct_s', 'ft_torque_util_70pct_s', 'ft_torque_util_80pct_s', 'ft_torque_util_90pct_s', 'ft_horsepower_util_50pct_s', 'ft_horsepower_util_60pct_s', 'ft_horsepower_util_70pct_s', 'ft_horsepower_util_80pct_s', 'ft_rpm_util_50pct_s', 'ft_rpm_util_60pct_s']
['vehicle_id', 'trip_id', 'datetime', 'velocity', 'accel_x', 'accel_y', 'accel_z', 'engine_coolant_temp', 'eng_load', 'fuel_level', 'iat', 'rpm', 'datetime_pst', 'wee

In [175]:
df_r.head(10)

,ft_torque_util_60pct_s,ft_torque_util_70pct_s,ft_torque_util_80pct_s,ft_torque_util_90pct_s,ft_horsepower_util_50pct_s,ft_horsepower_util_60pct_s,ft_horsepower_util_70pct_s,ft_horsepower_util_80pct_s,ft_rpm_util_50pct_s,ft_rpm_util_60pct_s,vehicle_id,week_start_Date
datetime_pst,,,,,,,,,,,,
2017-01-09 00:00:00-08:00,0.000000,751.350000,4970.366667,2.750000,5490.033333,202.250000,0.000000,0.000000,3359.466667,0.0,1000502,2017-01-09
2017-01-16 00:00:00-08:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1000502,2017-01-16
2017-01-23 00:00:00-08:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1000502,2017-01-23
2017-01-30 00:00:00-08:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1000502,2017-01-30
2017-02-06 00:00:00-08:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1000502,2017-02-06
2017-02-13 00:00:00-08:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1000502,2017-02-13
2017-02-20 00:00:00-08:00,1.483333,2533.483333,98.133333,0.000000,1312.916667,0.000000,0.000000,0.000000,32.100000,0.0,1000502,2017-02-20
2017-01-23 00:00:00-08:00,0.000000,3261.616667,3037.483333,0.000000,2433.283333,0.000000,0.000000,0.000000,0.000000,0.0,1000503,2017-01-23
2017-01-09 00:00:00-08:00,0.000000,3.716667,490.716667,2.066667,0.000000,0.083333,481.183333,15.233333,496.500000,0.0,1000504,2017-01-09


In [165]:
df_v.columns.tolist()

['vehicle_id',
 'trip_id',
 'datetime',
 'velocity',
 'accel_x',
 'accel_y',
 'accel_z',
 'engine_coolant_temp',
 'eng_load',
 'fuel_level',
 'iat',
 'rpm',
 'datetime_pst',
 'week_start_date',
 'year',
 'make',
 'Model',
 'drivetrain',
 'max_torque',
 'max_horsepower',
 'max_horsepower_rpm',
 'max_torque_rpm',
 'engine_displacement',
 'fuel_type',
 'fuel_tank_capacity',
 'fuel_economy_city',
 'fuel_economy_highway',
 'cylinders',
 'forced_induction',
 'device_generation',
 'active_horse_pow',
 'horse_pow',
 'torque_util',
 'rpm_util',
 'torque_util_per',
 'seconds',
 'ft_torque_util_60_pts',
 'ft_torque_util_60pct_s',
 'ft_torque_util_70pct_s',
 'ft_torque_util_80pct_s',
 'ft_torque_util_90pct_s',
 'ft_horsepower_util_50pct_s',
 'ft_horsepower_util_60pct_s',
 'ft_horsepower_util_70pct_s',
 'ft_horsepower_util_80pct_s',
 'ft_rpm_util_50pct_s',
 'ft_rpm_util_60pct_s']

In [166]:
for i,j in df_v.groupby("vehicle_id"):
    print(i)
    print(j.columns.tolist())

1000502
['vehicle_id', 'trip_id', 'datetime', 'velocity', 'accel_x', 'accel_y', 'accel_z', 'engine_coolant_temp', 'eng_load', 'fuel_level', 'iat', 'rpm', 'datetime_pst', 'week_start_date', 'year', 'make', 'Model', 'drivetrain', 'max_torque', 'max_horsepower', 'max_horsepower_rpm', 'max_torque_rpm', 'engine_displacement', 'fuel_type', 'fuel_tank_capacity', 'fuel_economy_city', 'fuel_economy_highway', 'cylinders', 'forced_induction', 'device_generation', 'active_horse_pow', 'horse_pow', 'torque_util', 'rpm_util', 'torque_util_per', 'seconds', 'ft_torque_util_60_pts', 'ft_torque_util_60pct_s', 'ft_torque_util_70pct_s', 'ft_torque_util_80pct_s', 'ft_torque_util_90pct_s', 'ft_horsepower_util_50pct_s', 'ft_horsepower_util_60pct_s', 'ft_horsepower_util_70pct_s', 'ft_horsepower_util_80pct_s', 'ft_rpm_util_50pct_s', 'ft_rpm_util_60pct_s']
1000503
['vehicle_id', 'trip_id', 'datetime', 'velocity', 'accel_x', 'accel_y', 'accel_z', 'engine_coolant_temp', 'eng_load', 'fuel_level', 'iat', 'rpm', 'dat

In [179]:
ordered_cols=['vehicle_id','week_start_date','ft_torque_util_60pct_s',
	'ft_torque_util_70pct_s',	'ft_torque_util_80pct_s',	'ft_torque_util_90pct_s',	'ft_horsepower_util_50pct_s',
    	'ft_horsepower_util_60pct_s',	'ft_horsepower_util_70pct_s',	'ft_horsepower_util_80pct_s',
        	'ft_rpm_util_50pct_s','ft_rpm_util_60pct_s']    
ordered_cols

['vehicle_id',
 'week_start_date',
 'ft_torque_util_60pct_s',
 'ft_torque_util_70pct_s',
 'ft_torque_util_80pct_s',
 'ft_torque_util_90pct_s',
 'ft_horsepower_util_50pct_s',
 'ft_horsepower_util_60pct_s',
 'ft_horsepower_util_70pct_s',
 'ft_horsepower_util_80pct_s',
 'ft_rpm_util_50pct_s',
 'ft_rpm_util_60pct_s']

In [182]:
ch=pd.read_csv("D:\experiments\dataEngineering\input_data\engine_features.csv")

In [183]:
ch[ordered_cols]

,vehicle_id,week_start_date,ft_torque_util_60pct_s,ft_torque_util_70pct_s,ft_torque_util_80pct_s,ft_torque_util_90pct_s,ft_horsepower_util_50pct_s,ft_horsepower_util_60pct_s,ft_horsepower_util_70pct_s,ft_horsepower_util_80pct_s,ft_rpm_util_50pct_s,ft_rpm_util_60pct_s
0,1000500,1/2/2017,30.966667,10498.183330,9008.083333,12.533333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1000500,1/9/2017,76.966667,46138.400000,32664.700000,80.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,1000500,1/16/2017,11.466667,2083.866667,618.966667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,1000500,1/23/2017,18.016667,20507.200000,10925.333330,56.983333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,1000500,1/30/2017,0.000000,6164.350000,1381.383333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,1000500,2/6/2017,0.000000,70.133333,889.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,1000500,2/13/2017,17.400000,5603.766667,104.466667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,1000500,2/20/2017,9.966667,19313.766670,23873.883330,21.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,1000500,2/27/2017,0.000000,96.283333,7963.500000,54.183333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,1000500,3/6/2017,1.533333,4756.533333,1121.583333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
